In [2]:
import numpy as np
import pandas as pd
import os

In [15]:
for i in os.walk(f'..{os.sep}data'):
    print(i)

('../data', ['shhs-study', 'isruc-sleep', 'shhs', 'Sleep-EDF', '.ipynb_checkpoints'], ['shhs2-hrv-5min-0.15.0.csv', 'shhs1-hrv-5min-0.15.0.csv', 'aggregated_sleep_data.csv', '.DS_Store', 'aggregateData2.ipynb', 'cleaned_sleep_stage_data.csv', 'raw_data.csv', 'shhs2-dataset-0.15.0.csv', 'aggregateData.ipynb', 'sleep_edf_raw.csv', 'isruc-sleep.tar.gz', 'shhs1-dataset-0.15.0.csv'])
('../data/shhs-study', [], ['aggregated_sleep_data.csv', 'cleaned_sleep_stage_data.csv', 'README.md'])
('../data/isruc-sleep', ['subgroupI', 'subgroupII', 'subgroupIII', '.ipynb_checkpoints'], ['.DS_Store', 'README.md'])
('../data/isruc-sleep/subgroupI', ['61', '95', '59', '92', '66', '50', '68', '57', '32', '35', '69', '56', '51', '58', '67', '93', '94', '60', '34', '33', '20', '18', '27', '9', '11', '7', '29', '16', '42', '89', '45', '73', '87', '80', '74', '6', '28', '17', '1', '10', '19', '26', '8', '21', '75', '81', '86', '72', '44', '43', '88', '38', '36', 'rec', '31', '91', '65', '62', '96', '100', '54',

In [3]:
df = pd.read_csv('aggregated_sleep_data.csv')
summary_isruc = pd.read_csv('isruc-sleep/subgroupI/scoring_summary_i.csv')

In [4]:
cleaned_df = pd.read_csv('cleaned_sleep_stage_data.csv')
cleaned_df = cleaned_df[['epoch', 'nsrrid',
       'sleep_stage']]
cleaned_df.columns = ['epoch', 'patient_id', 'stage']
cleaned_df 

,epoch,patient_id,stage
0,6960,200079,1
1,7260,200079,2
2,7560,200079,2
3,7860,200079,3
4,8160,200079,3
...,...,...,...
865185,31080,205677,2
865186,31380,205677,2
865187,31680,205677,0
865188,31980,205677,0


In [6]:
isruc1 = pd.read_csv(f'isruc-sleep{os.sep}subgroupI{os.sep}scoring_i.csv')
isruc2 = pd.read_csv(f'isruc-sleep{os.sep}subgroupII{os.sep}scoring_ii.csv')
isruc3 = pd.read_csv(f'isruc-sleep{os.sep}subgroupIII{os.sep}scoring_iii.csv')


In [7]:
isruc1 = isruc1[['epoch', 'stage', 'spo2', 'hr', 'events', 'bpos', 'txln',
       'txex', 'technote', 'patient_id', 'score_id']]

isruc1['patient_id'] += 2000

isruc2 = isruc2[['epoch', 'stage', 'spo2', 'hr', 'events', 'bpos', 'txln',
       'txex', 'technote', 'patient_id', 'score_id']]
isruc2['patient_id'] += 3000
isruc3 = isruc3[['epoch', 'stage', 'spo2', 'hr', 'events', 'bpos', 'txln',
       'txex', 'technote', 'patient_id', 'score_id']]
isruc2['patient_id'] += 4000

In [8]:
isruc = pd.concat([isruc1, isruc2, isruc3])
isruc = isruc[['epoch', 'patient_id','stage']]
isruc['epoch'] = isruc['epoch'].astype('int')
isruc

,epoch,patient_id,stage
0,1,2001,W
1,2,2001,W
2,3,2001,W
3,4,2001,W
4,5,2001,W
...,...,...,...
17773,792,10,W
17774,793,10,W
17775,794,10,W
17776,795,10,W


In [9]:
edf = pd.read_csv('sleep_edf_raw.csv')

In [10]:
edf[['epoch', 'patient_id', 'condition', 'EEG Fpz-Cz', 'EEG Pz-Oz',
       'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal',
       'Event marker']]
edf = edf[['epoch', 'patient_id','condition']]
edf.columns = ['epoch', 'patient_id', 'stage']

In [11]:
edf

,epoch,patient_id,stage
0,0,1,Sleep stage W
1,1,1,Sleep stage W
2,2,1,Sleep stage W
3,3,1,Sleep stage W
4,4,1,Sleep stage W
...,...,...,...
83454,1699,83,Sleep stage W
83455,1700,83,Sleep stage W
83456,1701,83,Sleep stage W
83457,1702,83,Sleep stage W


In [12]:
final_df = pd.concat([cleaned_df, isruc, edf])
final_df.to_csv('raw_data.csv')

In [13]:
final_df = final_df.replace('Sleep stage W', 'W')
final_df = final_df.replace('Sleep stage R', 'R')
final_df = final_df.replace('Sleep stage 1', '1')
final_df = final_df.replace('Sleep stage 2', '2')
final_df = final_df.replace('Sleep stage 3/4', '3')
final_df = final_df.replace('w', 'W')

final_df

,epoch,patient_id,stage
0,6960,200079,1
1,7260,200079,2
2,7560,200079,2
3,7860,200079,3
4,8160,200079,3
...,...,...,...
83454,1699,83,W
83455,1700,83,W
83456,1701,83,W
83457,1702,83,W


In [14]:
from sklearn.model_selection import train_test_split

X = final_df[['epoch','patient_id']]
y = final_df['stage']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
X_train.to_csv("X_train.csv")
X_test.to_csv("X_test.csv") 
y_train.to_csv("Y_train.csv")
y_test.to_csv("Y_test.csv")